# SqlAlchemy Relations Lab

### Introduction

So far we have seen how to set up sqlalchemy and use it to perform some single table queries.  It turns out that we can also use sqlalchemy to perform some of our relational queries.  

Let's get started.

### Loading our Data

Let's begin connecting to postgres, and then creating a database called `moes_bar_test`.  

```sql
create database moes_bar_test;
```

From there, run the `migrations/create_tables.sql` file against the database to create the necessary table.

We should also have a development database called `moes_bar`.  Probably best to start this fresh. 

Recreate the database.

```sql
drop database moes_bar;
create database moes_bar;
```

Then rhn the same `migrations/create_tables.sql` against our development database, `moes_bar`, and then run the `migrations/seed.sql` file against the database to seed the table with some initial data.

### Seeing our relations

Now let's dive directly into our flask shell.

```bash
flask shell
```

Log into the flask shell and run the following:

```python 
>>> bartender = db.session.query(Bartender).first()
>>> bartender.orders
```

Again, we can see that a bartender has many orders, and an order has one bartender.

We can see the related code for this in the Bartender class.

```python
# bartender.py

orders = relationship('Order', backref=backref('bartender'), cascade='all, delete-orphan')
```

And we should see that the related tests pass in the `test_bartender.py` file.

```bash
pytest tests/test_bartender.py
```

> We still will have one failure though.

Still let's see another way of defining this `bartender.orders` method.  Another way we can do this is by commenting out the current `orders` line, and replacing it with the following:
```python
# bartender.py
orders = relationship('Order', back_populates='bartender')
```

Then in the order.py file we have the following:

```python
bartender = relationship('Bartender', back_populates = 'orders')
```

So implementing the code this way, we need to add both a bartender method, and an orders method in the respective files.  And the back_populates just says that when we add an `order.bartender`, that order will automatically be seen in the `bartender.orders` method. 

If you look at the first test in the `test_bartender.py` file, you can see that this is exactly what's being tested.

```python
assert len(first_bartender.orders) == 2
```

### One to Many relationships

Ok, so now that we have the set up the bartender.orders and order.bartenders relationship, let's move onto relationship between drinks and orders.

> You will have to add **both** of the below methods to get your code in a working state.  If only one is implemented, your code will still be broken.

* `drink.orders` - add the drink.orders method using the back_populates argument

* `order.drink` - add the order.drink method also using back_populates.

Confirm that the related test in `test_order.py` works, and that the related test in `test_drink.py` works.

### Many to Many Relationships

Now that the one to many relationship is working, we should also see how to implement the many to many relationship.  One many to many relationship we have is that a drink has many bartenders and a bartender has many drinks.

* `drink.bartenders`

Let's take care of the drink's bartenders method first.

Remember that what links a drink to a bartender is our join table of orders.  Now take a look at how we implement the `drink.bartenders` method.

```python
# drink.py
bartenders = db.relationship('Bartender', secondary='orders')
```

So in the line above, we not only specify that the bartenders method should return Bartender instances, but also specify that to find the related bartenders, we need to query the orders table.  

Move this line to the `Drink` class, and see that the related test in `test_drink.py` passes -- it's the test that tests that a drink has many bartenders.

* bartender.drinks

Now a bartender has many drinks.  So we still need to add a `drinks` method to the bartender class.  Use the pattern above to add this method, and then run the `test_bartender.py` tests to confirm this passes.

### Add some tests

Next, we added some code for the Customer class.  It's your job to fill in the some of the code for the each of the tests -- notice that there is currently not even an assert statement in either test.

### Summary